# Lab 7:

In [99]:
import csv
import math
import numpy as np
from PIL import Image as pim

In [124]:
def calculate_features(input_array):
    features_array = np.zeros(input_array.shape, dtype=int)
    features_array[input_array != 255] = 1

    weight = np.sum(features_array)

    y_indices, x_indices = np.indices(features_array.shape)
    x_center_of_mass = np.sum(x_indices * features_array) / weight
    y_center_of_mass = np.sum(y_indices * features_array) / weight

    inertia_x = np.sum((y_indices - y_center_of_mass) ** 2 * features_array) / weight
    inertia_y = np.sum((x_indices - x_center_of_mass) ** 2 * features_array) / weight

    print("Char features: ")
    print(weight, x_center_of_mass, y_center_of_mass, inertia_x, inertia_y)

    return weight, x_center_of_mass, y_center_of_mass, inertia_x, inertia_y

In [115]:
def segment_letters(input_image):
    seg_array = np.sum(input_image == 0, axis=0)

    in_letter = False
    letter_bounds = []

    for i in range(len(seg_array)):
        if seg_array[i] > 0:
            if not in_letter:
                in_letter = True
                start = i
        else:
            if in_letter:
                in_letter = False
                end = i
                letter_bounds.append((start - 1, end))

    if in_letter:
        letter_bounds.append((start, len(seg_array)))

    return letter_bounds


In [116]:
def get_alphabet_info(kazakh_symbols) -> dict[chr, tuple]:
    def parse_tuple(string):
        return tuple(map(float, string.strip('()').split(',')))

    tuples_list = dict()
    with open('input/data.csv', 'r', encoding="utf-8") as file:
        reader = csv.DictReader(file, delimiter=';')
        i = 0
        for row in reader:
            weight = float(row['weight'])
            center_of_mass = parse_tuple(row['center_of_mass'])
            moment_of_inertia = parse_tuple(row['moment_of_inertia'])
            print(weight, center_of_mass, moment_of_inertia)
            tuples_list[kazakh_symbols[i]] = weight, *center_of_mass, *moment_of_inertia
            i += 1
            
    return tuples_list

In [117]:
def create_hypothesis(alphabet_info: dict[chr, tuple], target_features):
    def euclidean_distance(feature1, feature2):
        return math.sqrt(sum((a - b) ** 2 for a, b in zip(feature1, feature2)))

    distances = dict()
    for letter, features in alphabet_info.items():
        distance = euclidean_distance(target_features, features)
        distances[letter] = distance

    max_distance = max(distances.values())

    similarities = [(letter, round(1 - distance / max_distance, 2)) for letter, distance in distances.items()]

    return sorted(similarities, key=lambda x: x[1])


In [121]:
def get_phrase_from_hypothesis(input_array, bounds, symbols) -> str:
    alphabet_info = get_alphabet_info(symbols)

    res = []
    for start, end in bounds:

        letter_features = calculate_features(input_array[:, start: end])
        hypothesis = create_hypothesis(alphabet_info, letter_features)
        best_hypotheses = hypothesis[-1][0]
        res.append(best_hypotheses)
        
    
    return "".join(res)

In [122]:
def get_result(recognized_phrase, phrase):
    max_len = max(len(phrase), len(recognized_phrase))
    orig = phrase.ljust(max_len)
    detected = recognized_phrase.ljust(max_len)

    with open("output/result.txt", 'w', encoding="utf-8") as f:
        correct_letters = 0
        by_letter = ["has | got | correct"]
        for i in range(max_len):
            is_correct = orig[i] == detected[i]
            by_letter.append(f"{orig[i]}\t{detected[i]}\t{is_correct}")
            correct_letters += int(is_correct)
        f.write("\n".join([
            f"phrase:      {orig}",
            f"detected:    {detected}",
            f"correct:     {math.ceil(correct_letters / max_len * 100)}%\n\n"
        ]))
        f.write("\n".join(by_letter))

In [126]:
def main():
    kazakh_symbols = ["а", "ә", "б", "в", "г", "ғ", "д", "е", "ё",
                    "ж", "з", "и", "й", "к", "қ", "л", "м", "н",
                    "ң", "о", "ө", "п", "р", "с", "т", "у", "ұ",
                    "ү", "ф", "х", "һ", "ц", "ч", "ш", "щ", "ъ",
                    "ы", "і", "ь", "э", "ю", "я", ",", "."]

    input_phrase = "еңбек етсең ерінбей, тояды қарның тіленбей".replace(" ", "")

    input_image = np.array(pim.open(f'input/original_phrase.bmp').convert('L'))
    bounds = segment_letters(input_image)
    recognized_phrase = get_phrase_from_hypothesis(input_image, bounds, kazakh_symbols)
    print(recognized_phrase)

    get_result(recognized_phrase, input_phrase)

if __name__ == "__main__":
    main()

0.0881316098707403 (1.0399529964747356, 3.22972972972973) (174328.02447041674, 21914.79172771094)
0.08343125734430082 (1.0246768507638073, 2.9418331374853115) (154438.40523487286, 22225.021840621594)
0.08753315649867374 (1.1423519009725907, 2.6423519009725904) (171768.76604235132, 37650.709013173466)
0.09833333333333333 (1.135, 3.4988888888888887) (190094.86910740772, 24973.405825)
0.0701058201058201 (0.5502645502645502, 2.359126984126984) (110106.92898872525, 7472.45564233925)
0.08267195767195767 (0.6415343915343915, 2.7863756613756614) (125713.4848430369, 8516.86907736346)
0.07642998027613412 (1.1469428007889546, 2.828402366863905) (192446.54633941417, 35688.32114791344)
0.0799059929494712 (0.8466509988249119, 2.8501762632197414) (152547.3865377154, 16938.43905766492)
0.08059384941675504 (0.8658536585365854, 2.7205726405090136) (157205.5139179239, 19215.076740035685)
0.08796296296296297 (1.5354938271604939, 3.182098765432099) (259412.77034750843, 73201.03415066298)
0.0559845559845559